In [1]:
%matplotlib inline

import sys

import datacube
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import geopandas as gpd
import ipyleaflet as ipyl
import ipywidgets as ipyw
import json


import matplotlib as mpl
import matplotlib.colors
from matplotlib import cm
from matplotlib.colors import LinearSegmentedColormap
from colorspacious import cspace_converter
from collections import OrderedDict
from branca.colormap import linear

cmaps = OrderedDict()

In [6]:
shp_file_location = 'ABS_shapefiles/SA2_2016_AUST/SA2_2016_AUST.shp'

#Read the SA2 shapefile
gdf = gpd.read_file(shp_file_location)
gdf = gdf.rename(columns={'SA2_MAIN16':'SA2'})
gdf = gdf.set_index('SA2')
gdf = gdf.sort_index()
gdf = gdf.drop(columns=['SA2_5DIG16', 'SA2_NAME16', 'SA3_CODE16', 'SA3_NAME16', 'SA4_CODE16', 'SA4_NAME16',
                        'GCC_CODE16', 'GCC_NAME16', 'STE_CODE16', 'STE_NAME16', 'AREASQKM16'])

In [26]:
csv_file_location0 = 'ABS_csv_files/0_Cities_of_20000_or_more_Northern_Australia.csv'
csv_file_location1 = 'ABS_csv_files/1_well_serviced_towns_northern_australia.csv'
csv_file_location2 = 'ABS_csv_files/2_can_drive_to_a_town_in_an_hour.csv'
csv_file_location3 = 'ABS_csv_files/3_remote_areas_Northern_Australia.csv'

In [27]:
#0_Cities_of_20000_or_more_Northern_Australia.csv
df0 = pd.read_csv(csv_file_location0)
df0 = df0.set_index("SA2 code")
df0 = df0.sort_index()

SA2_index0 = df0.index
SA2_list0 = SA2_index0.tolist()
SA2_string0 = str(SA2_list0)
SA20 = SA2_string0.replace("[","").replace("]","").replace(" ","")
SA20 = SA20.split(",")

gdf0 = gdf[gdf.index.isin(SA20)]
new_gdf0 = gdf0.merge(df0, on=gdf0.index)
new_gdf0 = new_gdf0.rename(columns={'key_0':'SA2'})
new_gdf0 = new_gdf0.set_index('SA2')

In [28]:
#1_well_serviced_towns_northern_australia.csv
df1 = pd.read_csv(csv_file_location1)
df1 = df1.set_index("SA2 code")
df1 = df1.sort_index()

SA2_index1 = df1.index
SA2_list1 = SA2_index1.tolist()
SA2_string1 = str(SA2_list1)
SA21 = SA2_string1.replace("[","").replace("]","").replace(" ","")
SA21 = SA21.split(",")

gdf1 = gdf[gdf.index.isin(SA21)]
new_gdf1 = gdf1.merge(df1, on=gdf1.index)
new_gdf1 = new_gdf1.rename(columns={'key_0':'SA2'})
new_gdf1 = new_gdf1.set_index('SA2')

In [29]:
#2_can_drive_to_a_town_in_an_hour.csv
df2 = pd.read_csv(csv_file_location2)
df2 = df2.set_index("SA2 code")
df2 = df2.sort_index()

SA2_index2 = df2.index
SA2_list2 = SA2_index2.tolist()
SA2_string2 = str(SA2_list2)
SA22 = SA2_string2.replace("[","").replace("]","").replace(" ","")
SA22 = SA22.split(",")

gdf2 = gdf[gdf.index.isin(SA22)]
new_gdf2 = gdf2.merge(df2, on=gdf2.index)
new_gdf2 = new_gdf2.rename(columns={'key_0':'SA2'})
new_gdf2 = new_gdf2.set_index('SA2')

In [30]:
#3_remote_areas_Northern_Australia.csv
df3 = pd.read_csv(csv_file_location3)
df3 = df3.set_index("SA2 code")
df3 = df3.sort_index()

SA2_index3 = df3.index
SA2_list3 = SA2_index3.tolist()
SA2_string3 = str(SA2_list3)
SA23 = SA2_string3.replace("[","").replace("]","").replace(" ","")
SA23 = SA23.split(",")

gdf3 = gdf[gdf.index.isin(SA23)]
new_gdf3 = gdf3.merge(df3, on=gdf3.index)
new_gdf3 = new_gdf3.rename(columns={'key_0':'SA2'})
new_gdf3 = new_gdf3.set_index('SA2')

In [31]:
#Converting the data to json
data0 = json.loads(new_gdf0.to_json())
data1 = json.loads(new_gdf1.to_json())
data2 = json.loads(new_gdf2.to_json())
data3 = json.loads(new_gdf3.to_json())

map = ipyl.Map(center=[-19.3, 134], zoom=5)

label = ipyw.Label(layout=ipyw.Layout(width='100%'))

#0_Cities_of_20000_or_more_Northern_Australia.csv
for feature in data0['features']:
    feature['properties']['style'] = {
        'color': 'green',
        'weight': 1,
        'fillColor': 'green',
        'fillOpacity': 0.5
    }
layer0 = ipyl.GeoJSON(data=data0, hover_style={'fillColor': 'blue'})


#1_well_serviced_towns_northern_australia.csv
for feature in data1['features']:
    feature['properties']['style'] = {
        'color': 'yellow',
        'weight': 1,
        'fillColor': 'yellow',
        'fillOpacity': 0.5
    }
layer1 = ipyl.GeoJSON(data=data1, hover_style={'fillColor': 'blue'})

#2_can_drive_to_a_town_in_an_hour.csv
for feature in data2['features']:
    feature['properties']['style'] = {
        'color': 'red',
        'weight': 1,
        'fillColor': 'red',
        'fillOpacity': 0.5
    }
layer2 = ipyl.GeoJSON(data=data2, hover_style={'fillColor': 'blue'})


#3_remote_areas_Northern_Australia.csv
for feature in data3['features']:
    feature['properties']['style'] = {
        'color': 'grey',
        'weight': 1,
        'fillColor': 'grey',
        'fillOpacity': 0.5
    }
layer3 = ipyl.GeoJSON(data=data3, hover_style={'fillColor': 'blue'})


def click_handler(event=None, feature=None, id=None, properties=None):
    label.value = str(properties['Name'])
    
    
layer0.on_hover(click_handler)
map.add_layer(layer0)

layer1.on_hover(click_handler)
map.add_layer(layer1)

layer2.on_hover(click_handler)
map.add_layer(layer2)

layer3.on_hover(click_handler)
map.add_layer(layer3)

ipyw.VBox([map, label])